In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st

from sklearn.model_selection import (train_test_split, cross_validate,
                                     GridSearchCV, RandomizedSearchCV)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, f1_score, classification_report

In [28]:
df = sns.load_dataset("titanic").copy()

In [29]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [30]:
df = df[~df["survived"].isna()].copy()
y = df["survived"].astype(int)

In [31]:
num_features = ["age", "fare", "sibsp", "parch", "pclass"]
cat_features = ["sex", "embarked", "class", "alone"]

In [32]:
X = df[num_features + cat_features].copy()

In [33]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print(f"Train: {X_train.shape}, Test: {X_test.shape}, Pos-Rate Train: {y_train.mean():.3f}, Pos-Rate overall: {y.mean():.3f}")

Train: (712, 9), Test: (179, 9), Pos-Rate Train: 0.383, Pos-Rate overall: 0.384


### Aufteilen in Trainings- und Testdaten

Hier wird der Datensatz in einen Trainings- und einen Testteil getrennt.  
Die Aufteilung ist notwendig, um eine unabhängige Schätzung der Generalisierungsleistung zu erhalten – also zu prüfen, wie gut ein Modell auf neuen, nicht im Training gesehenen Daten funktioniert.

Der Parameter `test_size=0.2` legt fest, dass 20 % der Daten für den Test und 80 % für das Training verwendet werden.

Mit `stratify=y` wird sichergestellt, dass das Klassenverhältnis (z. B. Anteil Überlebender und Nicht-Überlebender) im Trainings- und Testset gleich bleibt.  
Das ist besonders wichtig bei unbalancierten Datensätzen, weil sich sonst zufällige Verschiebungen in den Anteilen direkt auf die Modellbewertung auswirken würden.

Der `random_state=42` sorgt dafür, dass die Aufteilung reproduzierbar bleibt – bei jeder Ausführung werden dieselben Beobachtungen in Train- bzw. Testset gezogen.

Die `print`-Ausgabe fasst zusammen:
- die Form der Trainings- und Testdaten (Anzahl Beobachtungen × Merkmale),
- die durchschnittliche Zielvariable im Training und insgesamt.  
Ein Vergleich dieser beiden Werte zeigt, ob die Stratifizierung korrekt funktioniert hat.


In [34]:
numeric_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, num_features),
        ("cat", categorical_pipe, cat_features),
    ],
    remainder="drop"
)

### Vorverarbeitung mit Pipelines und ColumnTransformer

Dieser Abschnitt definiert die systematische Vorverarbeitung der Merkmale.  
Da der Datensatz sowohl numerische als auch kategoriale Variablen enthält, werden zwei Teilpipelines erstellt und anschließend mit einem `ColumnTransformer` kombiniert.

Die **numerische Pipeline** (`numeric_pipe`) behandelt alle kontinuierlichen Merkmale:
- Fehlende Werte werden durch den Median ersetzt (`SimpleImputer(strategy="median")`), um Ausreißerrobustheit zu gewährleisten.  
- Anschließend werden die Werte standardisiert (`StandardScaler()`), sodass jedes Merkmal Mittelwert 0 und Standardabweichung 1 hat.  
  Das ist insbesondere für Modelle wie die logistische Regression notwendig, deren Regularisierung von der Skalierung abhängt.

Die **kategoriale Pipeline** (`categorical_pipe`) verarbeitet alle nominalen Variablen:
- Fehlende Werte werden durch die häufigste Ausprägung ersetzt (`SimpleImputer(strategy="most_frequent")`).  
- Danach werden die Kategorien mit `OneHotEncoder()` in numerische Dummy-Variablen überführt.  
  Der Parameter `handle_unknown="ignore"` sorgt dafür, dass unbekannte Kategorien im Testdatensatz keine Fehler verursachen – ein zentraler Punkt für Reproduzierbarkeit.  
  Mit `sparse_output=False` wird ein reguläres NumPy-Array zurückgegeben, was spätere Verarbeitungsschritte vereinfacht. Mit `sparse_output=False` wird ein reguläres NumPy-Array zurückgegeben.  
Das ist hier sinnvoll, weil der nachgelagerte Klassifikator (`LogisticRegression` mit dem Solver `'lbfgs'`) dichte Eingaben erwartet.  
Würde der Encoder eine Sparse-Matrix liefern, müsste entweder ein anderer Solver (`'saga'` oder `'liblinear'`) verwendet oder die Matrix manuell mit `.toarray()` in ein dichtes Format umgewandelt werden.  
Beides würde den Workflow unnötig verkomplizieren und etwas Rechenzeit kosten.  
Zudem lässt sich die dichte Ausgabe leichter inspizieren oder in einen DataFrame überführen.

Der **ColumnTransformer** (`preprocess`) führt beide Teilpipelines zusammen:
- Die in `num_features` angegebenen Spalten durchlaufen die numerische Pipeline,  
- die in `cat_features` genannten Spalten die kategoriale Pipeline.  
- Mit `remainder="drop"` werden alle übrigen Spalten verworfen.

Damit entsteht eine einheitliche, reproduzierbare Vorverarbeitung, die später in die Gesamtpipeline mit dem Modell integriert wird.  
Dadurch wird sichergestellt, dass **alle Datenvorbereitungsschritte identisch auf Trainings- und Testdaten angewendet** werden und kein Daten-Leakage entsteht.


### Alternative Schritte und Varianten in der Vorverarbeitung

Die hier verwendeten Schritte – Medianimputation, Standardisierung und One-Hot-Encoding – sind typische Standardentscheidungen.  
Je nach Datensatzstruktur oder Modellwahl sind jedoch auch andere Verfahren sinnvoll.  
Wichtiger als die konkrete Wahl ist, dass die **Vorverarbeitung zum Datentyp und zum Modell** passt.

**Mögliche Alternativen und Erweiterungen:**

- **Imputation**
  - `SimpleImputer(strategy="mean")` für symmetrisch verteilte numerische Daten.  
  - `KNNImputer()` oder `IterativeImputer()` bei komplexeren Mustern fehlender Werte.
  - Bei Textdaten oder IDs ist Imputation oft nicht sinnvoll – fehlende Werte werden dann besser als eigene Kategorie behandelt.

- **Skalierung und Transformation**
  - `MinMaxScaler()` wenn Merkmale auf ein festes Intervall (z. B. [0, 1]) gebracht werden sollen.  
  - `RobustScaler()` bei Ausreißern, da er Perzentile statt Mittelwert und Standardabweichung nutzt.  
  - `PowerTransformer()` oder `QuantileTransformer()` bei stark schiefen Verteilungen.

- **Kodierung kategorialer Variablen**
  - `OrdinalEncoder()` wenn die Kategorien eine natürliche Reihenfolge haben (z. B. Bildungsniveau).  
  - `TargetEncoder` oder `LeaveOneOutEncoder` (aus `category_encoders`) bei sehr vielen Ausprägungen oder hochkardinalen Merkmalen.  
  - Bei Textmerkmalen: `CountVectorizer` oder `TfidfVectorizer` (aus `sklearn.feature_extraction.text`).

- **Feature-Selektion und -Erzeugung**
  - `PolynomialFeatures()` oder `InteractionTermTransformer` für Modelle, die keine nichtlinearen Beziehungen selbst abbilden können.  
  - `SelectKBest` oder `VarianceThreshold` zur Reduktion der Merkmalsanzahl.

**Wichtige Grundregel:**  
Die gesamte Vorverarbeitung muss Teil der Pipeline sein, damit sie während Cross-Validation und späterem Tuning  
**nur auf Trainingsdaten** angewendet wird. Nur so bleibt die Bewertung unverzerrt und reproduzierbar.


### Wann und wie man alternative Vorverarbeitungsschritte in Pipelines integriert

Je nach Datentyp, Modell und Zielsetzung kann die Vorverarbeitung in einer Pipeline stark variieren.  
Die folgenden Beispiele zeigen typische Situationen, in denen andere Transformer sinnvoll sind –  
immer mit dem Ziel, dass alle Schritte automatisch innerhalb von Cross-Validation oder Hyperparameter-Tuning korrekt ausgeführt werden.

### 1. Nichtlineare Beziehungen mit PolynomialFeatures

Lineare Modelle wie die logistische Regression können nur gerade Zusammenhänge zwischen Merkmalen und Ziel abbilden.  
Wenn ein Effekt gekrümmt oder multiplikativ ist, lässt sich dieser Zusammenhang mit zusätzlichen Polynom- oder Interaktionstermen besser modellieren.  
`PolynomialFeatures(degree=2)` erzeugt aus bestehenden numerischen Merkmalen alle Quadrate und Kreuzprodukte bis zum angegebenen Grad.  
Nach der Transformation werden die neuen Merkmale standardisiert, bevor das Modell trainiert wird.

```python
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ("poly", PolynomialFeatures(degree=2, include_bias=False)),
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=2000))
])

### 2. Merkmalsreduktion mit SelectKBest

Wenn viele Merkmale vorliegen, ist nicht jedes gleich informativ.  
`SelectKBest` wählt die *k* Variablen mit dem stärksten Zusammenhang zur Zielgröße.  
Das reduziert Rauschen, spart Rechenzeit und kann Überanpassung verhindern.  
Die Auswahl erfolgt über eine Bewertungsfunktion wie den F-Test (`f_classif`) für Klassifikationsaufgaben.

```python
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("select", SelectKBest(score_func=f_classif, k=10)),
    ("clf", LogisticRegression(max_iter=2000))
])

### 3. Textdaten mit TfidfVectorizer

Textdaten müssen zunächst in numerische Form gebracht werden, bevor sie in ein Modell eingehen können.  
`TfidfVectorizer` wandelt Wörter in gewichtete Wortmerkmale um: häufige, aber wenig aussagekräftige Wörter werden abgewertet, seltene, aber charakteristische stärker gewichtet.  
Dadurch kann ein lineares Modell wie die logistische Regression direkt auf Textdaten trainiert werden.

```python
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ("tfidf", TfidfVectorizer(max_features=1000, ngram_range=(1, 2))),
    ("clf", LogisticRegression(max_iter=2000))
])

### 4. Hochkardinale Kategorien mit TargetEncoder

Wenn kategoriale Variablen sehr viele mögliche Werte besitzen (z. B. Beruf, Produkt-ID oder Land),  
führt One-Hot-Encoding zu sehr vielen Spalten.  
Der `TargetEncoder` ersetzt jede Kategorie durch den durchschnittlichen Zielwert (z. B. Überlebenswahrscheinlichkeit).  
So bleibt die Dimension klein, und die Information über den Zusammenhang zum Ziel erhalten.  
Die Kodierung muss Teil der Pipeline sein, damit sie während Cross-Validation korrekt nur auf Trainingsdaten berechnet wird.

```python
from category_encoders import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", TargetEncoder())
])

preprocess = ColumnTransformer([
    ("cat", cat_pipe, ["occupation"])
])

pipe = Pipeline([
    ("prep", preprocess),
    ("clf", LogisticRegression(max_iter=2000))
])

In [35]:
base_clf = LogisticRegression(max_iter=2000, solver="lbfgs")

pipe = Pipeline([
    ("prep", preprocess),
    ("clf", base_clf)
])

### Aufbau der Modell-Pipeline

Hier wird das eigentliche Modell – die logistische Regression – mit der zuvor definierten Vorverarbeitung (`preprocess`) zu einer vollständigen Pipeline kombiniert.

- **`base_clf = LogisticRegression(max_iter=2000, solver="lbfgs")`**  
  Erstellt das Basismodell.  
  - Der Parameter `max_iter=2000` sorgt dafür, dass die Optimierung (Gradientenverfahren) auch bei vielen Merkmalen sicher konvergiert.  
  - Der Solver `'lbfgs'` ist ein stabiler und effizienter Optimierungsalgorithmus für L2-regulierte logistische Regression und eignet sich besonders bei dichten Eingaben (wie hier nach `sparse_output=False` im Encoder).

- **Pipeline-Struktur**  
  ```python
  pipe = Pipeline([
      ("prep", preprocess),
      ("clf", base_clf)
  ])


### Baseline-Modell: Training und erste Evaluation

Nachdem die Pipeline definiert ist, wird sie nun mit den Trainingsdaten trainiert und auf dem Testset bewertet.  
Das Ziel dieses Schritts ist es, eine **Baseline-Leistung** zu ermitteln – also eine erste, einfache Referenz,  
gegen die spätere Verbesserungen durch Hyperparameter-Tuning gemessen werden können.

```python
pipe.fit(X_train, y_train)
baseline_acc = pipe.score(X_test, y_test)
print(f"Baseline Test Accuracy: {baseline_acc:.3f}")


In [36]:
pipe.fit(X_train, y_train)
baseline_acc = pipe.score(X_test, y_test)
print(f"Baseline Test Accuracy: {baseline_acc:.3f}")

y_pred_base = pipe.predict(X_test)
print("Baseline Test F1-macro:", f1_score(y_test, y_pred_base, average="macro"))

Baseline Test Accuracy: 0.804
Baseline Test F1-macro: 0.7879218712975187


In [54]:
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_base):.3f}")
print(f"Test F1-macro: {f1_score(y_test, y_pred_base, average='macro'):.3f}")
print("\nDetailed classification report:")
print(classification_report(y_test, y_pred_base))

Test Accuracy: 0.804
Test F1-macro: 0.788

Detailed classification report:
              precision    recall  f1-score   support

           0       0.82      0.88      0.85       110
           1       0.78      0.68      0.73        69

    accuracy                           0.80       179
   macro avg       0.80      0.78      0.79       179
weighted avg       0.80      0.80      0.80       179



### Bedeutung von `average="macro"` beim F1-Score

Der F1-Score kombiniert **Precision** (Anteil korrekter positiver Vorhersagen)  
und **Recall** (Anteil korrekt erkannter positiver Fälle) zu einem harmonischen Mittel.  
Bei mehr als zwei Klassen oder bei unbalancierten Daten muss angegeben werden, wie die einzelnen Klassen gewichtet werden sollen.  
Das geschieht über den Parameter `average`.

**Wichtige Varianten:**
- `average="macro"`  
  → Der F1-Score wird **für jede Klasse separat** berechnet und anschließend **gleich gewichtet gemittelt**.  
  Alle Klassen zählen also gleich stark, **unabhängig von ihrer Häufigkeit**.  
  Dadurch ist die Kennzahl empfindlich für Klassenungleichgewicht, was sie für faire Vergleiche nützlich macht.
  
- `average="weighted"`  
  → Wie `"macro"`, aber jede Klasse wird **nach ihrer Häufigkeit gewichtet**.  
  Häufige Klassen beeinflussen das Ergebnis stärker.

- `average="micro"`  
  → Berechnet eine **globale Precision und Recall** über alle Klassen hinweg,  
  also so, als wären alle Beobachtungen Teil einer einzigen großen Binärklassifikation.

In unserem Fall (`average="macro"`) erhält jede Klasse – z. B. „überlebt“ und „nicht überlebt“ – denselben Einfluss auf den Gesamtscore.  
Das verhindert, dass die häufigere Klasse die Bewertung dominiert und ist daher besser geeignet,  
wenn Klassenverteilungen ungleich sind.


In [37]:
scoring = "f1_macro" 

In [38]:
rf = RandomForestClassifier(random_state=42)
rf_pipe = Pipeline([
    ("prep", preprocess),
    ("clf", rf)
])

In [39]:
param_grid_rf = {
    "clf__n_estimators": [100, 200, 300],
    "clf__max_features": ["sqrt", "log2", None],
    "clf__min_samples_leaf": [1, 2, 5]
}

In [40]:
rf_gcv = GridSearchCV(
    rf_pipe, param_grid_rf, cv=5, scoring=scoring, verbose=4
)

In [41]:
rf_gcv.fit(X_train, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5] END clf__max_features=sqrt, clf__min_samples_leaf=1, clf__n_estimators=100;, score=0.775 total time=   0.0s
[CV 2/5] END clf__max_features=sqrt, clf__min_samples_leaf=1, clf__n_estimators=100;, score=0.688 total time=   0.0s
[CV 3/5] END clf__max_features=sqrt, clf__min_samples_leaf=1, clf__n_estimators=100;, score=0.815 total time=   0.0s
[CV 4/5] END clf__max_features=sqrt, clf__min_samples_leaf=1, clf__n_estimators=100;, score=0.837 total time=   0.0s
[CV 5/5] END clf__max_features=sqrt, clf__min_samples_leaf=1, clf__n_estimators=100;, score=0.781 total time=   0.0s
[CV 1/5] END clf__max_features=sqrt, clf__min_samples_leaf=1, clf__n_estimators=200;, score=0.762 total time=   0.1s
[CV 2/5] END clf__max_features=sqrt, clf__min_samples_leaf=1, clf__n_estimators=200;, score=0.664 total time=   0.1s
[CV 3/5] END clf__max_features=sqrt, clf__min_samples_leaf=1, clf__n_estimators=200;, score=0.807 total time=   0.1s
[C

,estimator,Pipeline(step...m_state=42))])
,param_grid,"{'clf__max_features': ['sqrt', 'log2', ...], 'clf__min_samples_leaf': [1, 2, ...], 'clf__n_estimators': [100, 200, ...]}"
,scoring,'f1_macro'
,n_jobs,None
,refit,True
,cv,5
,verbose,4
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('cat', ...)]"


In [42]:
rf_best = rf_gcv.best_estimator_

In [43]:
rf_test_pred = rf_best.predict(X_test)

In [44]:
print("\nRandomForest – best params:", rf_gcv.best_params_)
print("RandomForest – Test F1-macro:",
      f1_score(y_test, rf_test_pred, average="macro"))
print("RandomForest – Test Accuracy:",
      accuracy_score(y_test, rf_test_pred))


RandomForest – best params: {'clf__max_features': 'sqrt', 'clf__min_samples_leaf': 2, 'clf__n_estimators': 300}
RandomForest – Test F1-macro: 0.7902536531568789
RandomForest – Test Accuracy: 0.8100558659217877


In [55]:
print("\nDetailed classification report:")
print(classification_report(y_test, rf_test_pred))


Detailed classification report:
              precision    recall  f1-score   support

           0       0.81      0.91      0.85       110
           1       0.82      0.65      0.73        69

    accuracy                           0.81       179
   macro avg       0.81      0.78      0.79       179
weighted avg       0.81      0.81      0.81       179



In [46]:
param_dist_rf = {
    "clf__n_estimators": st.randint(200, 601),     # 200–600 Bäume
    "clf__max_features": ["sqrt", "log2", 0.3, 0.5, 0.7],  # Anteil Merkmale pro Split
    "clf__min_samples_leaf": st.randint(1, 16),    # 1–15 Beobachtungen pro Blatt
}

In [47]:
rf_rcv = RandomizedSearchCV(
    estimator=rf_pipe, 
    param_distributions=param_dist_rf,
    n_iter=30,                # Anzahl zufälliger Kombinationen (Tuning-Budget)
    cv=5,                     # Cross-Validation
    scoring=scoring,          # z. B. "f1_macro"
    verbose=4,                # Fortschrittsanzeige
    random_state=42,
    n_jobs=1,                 # n_jobs=1, damit verbose-Ausgabe sichtbar bleibt
    return_train_score=True
)

In [48]:
rf_rcv.fit(X_train, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV 1/5] END clf__max_features=0.5, clf__min_samples_leaf=13, clf__n_estimators=470;, score=(train=0.811, test=0.798) total time=   0.3s
[CV 2/5] END clf__max_features=0.5, clf__min_samples_leaf=13, clf__n_estimators=470;, score=(train=0.812, test=0.669) total time=   0.3s
[CV 3/5] END clf__max_features=0.5, clf__min_samples_leaf=13, clf__n_estimators=470;, score=(train=0.794, test=0.804) total time=   0.3s
[CV 4/5] END clf__max_features=0.5, clf__min_samples_leaf=13, clf__n_estimators=470;, score=(train=0.789, test=0.797) total time=   0.3s
[CV 5/5] END clf__max_features=0.5, clf__min_samples_leaf=13, clf__n_estimators=470;, score=(train=0.803, test=0.793) total time=   0.3s
[CV 1/5] END clf__max_features=0.3, clf__min_samples_leaf=8, clf__n_estimators=388;, score=(train=0.848, test=0.778) total time=   0.2s
[CV 2/5] END clf__max_features=0.3, clf__min_samples_leaf=8, clf__n_estimators=388;, score=(train=0.827, test=0.686) 

,estimator,Pipeline(step...m_state=42))])
,param_distributions,"{'clf__max_features': ['sqrt', 'log2', ...], 'clf__min_samples_leaf': <scipy.stats....00122724477D0>, 'clf__n_estimators': <scipy.stats....001226F244050>}"
,n_iter,30
,scoring,'f1_macro'
,n_jobs,1
,refit,True
,cv,5
,verbose,4
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [49]:
print("RandomizedSearchCV – best params:", rf_rcv.best_params_)
print(f"RandomizedSearchCV – mean CV F1-macro: {rf_rcv.best_score_:.3f}")

RandomizedSearchCV – best params: {'clf__max_features': 0.5, 'clf__min_samples_leaf': 2, 'clf__n_estimators': 589}
RandomizedSearchCV – mean CV F1-macro: 0.808


In [50]:
best_rf = rf_rcv.best_estimator_

In [51]:
y_pred = best_rf.predict(X_test)

print(f"Test Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print(f"Test F1-macro: {f1_score(y_test, y_pred, average='macro'):.3f}")
print("\nDetailed classification report:")
print(classification_report(y_test, y_pred))

Test Accuracy: 0.804
Test F1-macro: 0.785

Detailed classification report:
              precision    recall  f1-score   support

           0       0.80      0.90      0.85       110
           1       0.80      0.65      0.72        69

    accuracy                           0.80       179
   macro avg       0.80      0.78      0.78       179
weighted avg       0.80      0.80      0.80       179

